##### doc2vec

In [1]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm

In [2]:
data = pd.read_csv("data_토큰화.csv", encoding='utf-8-sig')
data.head()

,Unnamed: 0,index,time,doc,url,token,text_noun
0,0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775,"['드라마', '설강화', '배우', '김혜윤', '배우', '김혜윤']","['드라마', '설강화', '배우', '김혜윤', '배우', '김혜윤']"
1,1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031,"['인스타', '트위터', '댓글', '프루덴셜', '센터']","['인스타', '트위터', '댓글', '프루덴셜', '센터']"
2,2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875,"['사진', '모음', '사진', '모음', '사진', '모음', '사진', '모음']","['사진', '모음', '사진', '모음', '사진', '모음', '사진', '모음']"
3,3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800,['게시판'],['게시판']
4,4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...,"['스토리', '포에버', '팔로워', '팔로잉', '게시', '물']","['스토리', '포에버', '팔로워', '팔로잉', '게시', '물']"


In [3]:
#필요없는 컬럼 삭제
del data['Unnamed: 0']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7089 entries, 0 to 7088
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7089 non-null   int64 
 1   time       7089 non-null   object
 2   doc        7089 non-null   object
 3   url        7089 non-null   object
 4   token      7089 non-null   object
 5   text_noun  7089 non-null   object
dtypes: int64(1), object(5)
memory usage: 332.4+ KB


In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from gensim.test.utils import common_texts

In [5]:
import ast
token = []
token_noun = []
for i in range(len(data['token'])):
    token.append(ast.literal_eval(data['token'][i]))
    token_noun.append(ast.literal_eval(data['text_noun'][i]))

In [6]:
data['token'] = token
data['text_noun'] = token_noun

In [7]:
documents = [TaggedDocument(word, [i]) for i, word in enumerate(data['token'])]

In [8]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=20,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=3,
                dm =1)
  
model.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=10)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [9]:
data['token'][0]

['드라마', '설강화', '배우', '김혜윤', '배우', '김혜윤']

In [10]:
vector = model.infer_vector(data['token'][0])

In [11]:
len(data['token'])

7089

In [19]:
model.random.seed(9999)
rv = []
for i in tqdm(data['token']):
    vector = model.infer_vector(i)
    rv.append(vector)

100%|██████████| 7089/7089 [00:04<00:00, 1506.53it/s]


In [20]:
len(rv)

7089

In [21]:
data['review_vec'] = rv

In [22]:
data.rename(columns = {'Unnamed: 0': 'doc_num'}, inplace=True)

In [16]:
#data.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1'], axis=1, inplace=True)

In [23]:
data

,index,time,doc,url,token,text_noun,review_vec
0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775,"[드라마, 설강화, 배우, 김혜윤, 배우, 김혜윤]","[드라마, 설강화, 배우, 김혜윤, 배우, 김혜윤]","[-0.03705043, 0.0030446704, 0.10498903, 0.0117..."
1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031,"[인스타, 트위터, 댓글, 프루덴셜, 센터]","[인스타, 트위터, 댓글, 프루덴셜, 센터]","[-0.01738648, 0.0005801313, 0.021741647, 0.009..."
2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875,"[사진, 모음, 사진, 모음, 사진, 모음, 사진, 모음]","[사진, 모음, 사진, 모음, 사진, 모음, 사진, 모음]","[0.0061358213, 0.053706825, 0.059647392, 0.033..."
3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800,[게시판],[게시판],"[-0.04382301, 0.020393278, 0.06901242, 0.05213..."
4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...,"[스토리, 포에버, 팔로워, 팔로잉, 게시, 물]","[스토리, 포에버, 팔로워, 팔로잉, 게시, 물]","[-0.042341266, 0.04884327, 0.059883893, 0.0100..."
...,...,...,...,...,...,...,...
7084,7700,2022. 10. 12. 15:31,크리스탈 폴로 셔츠 포니 가방 캐주얼룩 완성 걸스픽 크리스탈 폴로 ...,https://blog.naver.com/gguu1029/222898345842,"[폴로, 셔츠, 포니, 가방, 얼룩, 완성, 걸스, 픽, 폴로, 셔츠, 포니, 가방...","[폴로, 셔츠, 포니, 가방, 얼룩, 완성, 걸스, 픽, 폴로, 셔츠, 포니, 가방...","[-0.71085846, 0.079961635, 0.8093155, -0.54867..."
7085,7701,2022. 7. 16. 9:37,스타일링 따라하고 싶은 셀럽들의 폴로 랄프로렌 셔츠 스타일링 크리스탈 기은세 꾸리...,https://blog.naver.com/kims718/222813529091,"[스타, 일링, 셀럽, 폴로, 랄프, 로렌, 셔츠, 스타, 일링, 기은세, 꾸리, ...","[스타, 일링, 셀럽, 폴로, 랄프, 로렌, 셔츠, 스타, 일링, 기은세, 꾸리, ...","[-0.3739973, 0.17031088, -0.02671735, -0.53713..."
7086,7702,2022. 9. 18. 6:40,게시판 크리스탈 정수정 근황 좋아보여요 광고실행사 배우 정수정 및 크리스탈...,https://blog.naver.com/xkyzscczg/222875005243,"[게시판, 정수정, 근황, 좋다, 광고, 실, 행사, 배우, 정수정, 님근황, 해바...","[게시판, 정수정, 근황, 광고, 실, 행사, 배우, 정수정, 님근황, 해바라기, ...","[-0.4879199, 0.2553368, 0.3971704, -0.03815675..."
7087,7704,2022. 10. 8. 11:20,스타패션 크리스탈 정수정 인스타그램 스트라이프 럭비 티셔츠 폴로 랄프로렌 ...,https://blog.naver.com/in_korean_style/2228947...,"[스타, 패션, 정수정, 스트라이프, 럭비, 티셔츠, 폴로, 랄프, 로렌, 안녕하다...","[스타, 패션, 정수정, 스트라이프, 럭비, 티셔츠, 폴로, 랄프, 로렌, 스트라이...","[-0.21006502, 0.2101962, 0.21272005, 0.1941829..."


In [24]:
data.to_pickle('data_벡터화(토큰).pkl')
data.to_csv("data_벡터화(토큰).csv", encoding='utf-8-sig')